In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import torch
import math

from modules.util.util import importstr
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [3]:
def run(app, *argv):    
    torch.cuda.empty_cache()
    
    argv = list(argv)
    argv.insert(0, '--num-workers=4')
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}.main()".format(app, argv))

In [4]:
# we use this to test if the bounding box dimension is valid for our unet
def test_num(num, depth):
    offset = int(math.ceil(num*.33))
    curr_width = width = num - offset
    for i in range(depth-1):
        curr_width = math.floor(curr_width / 2)
    for i in range(depth-1):
        curr_width = curr_width * 2
    return width == curr_width, repr([width, curr_width])

In [5]:
experiment_epochs = 3
unet_depth=3
val_cadence = 1
batch_size = 32
recall_priority = 0
pad_types = ['zero', 'reflect', 'replicate']
windows = [None, 'lung', 'mediastinal', 'dist']

bbox_dim = 246

In [6]:
for num in [60,108,144,198,246,288,323]:
    works, dim = test_num(num, unet_depth)
    assert works, repr(dim)

Cache all the different sizes of bounding box

In [7]:
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '60', '60')
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '108', '108')
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '144', '144')
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '198', '198')
run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '246', '246')
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '288', '288')
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '323', '323')

2020-11-14 19:02:16,930 INFO     pid:54362 nb:006:run Running: prepcache.CovidPrepCacheApp(['--num-workers=4', '--width-irc', '7', '246', '246']).main()
2020-11-14 19:02:17,872 INFO     pid:54362 prepcache:039:main Starting CovidPrepCacheApp, Namespace(batch_size=1024, num_workers=4, width_irc=['7', '246', '246'])
100%|██████████| 2/2 [01:20<00:00, 40.47s/it]
2020-11-14 19:03:38,881 INFO     pid:54362 nb:011:run Finished: prepcache.CovidPrepCacheApp.['--num-workers=4', '--width-irc', '7', '246', '246'].main()


# Bounding Box Test

In [ ]:
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--run-name=bbox60')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--width-irc', '7', '108', '108', '--run-name=bbox108')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--width-irc', '7', '144', '144', '--run-name=bbox144')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--width-irc', '7', '198', '198', '--run-name=bbox198')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--width-irc', '7', '246', '246', '--run-name=bbox246')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--width-irc', '7', '288', '288', '--run-name=bbox288')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--width-irc', '7', '323', '323', '--run-name=bbox323')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

# Augmentation Test

In [9]:
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-flip', '--run-name=flip aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-offset', '--run-name=offset aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-scale', '--run-name=scale aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-rotate', '--run-name=rotate aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-noise', '--run-name=noise aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augmented', '--run-name=all aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

In [10]:
# increased range of scale from .2 to .3 and range of offset from .1 to .3
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-offset', '--run-name=larger offset aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-scale', '--run-name=larger scale aug', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

# Window Test

In [9]:
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', f'--ct-window={windows[1]}', 
    '--augmented', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}', '--run-name=lung window')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', f'--ct-window={windows[2]}', 
    '--augmented', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}', '--run-name=mediastinal window')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

# Padding Test

In [7]:
# run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', f'--ct-window={windows[1]}', 
#     '--augmented', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}', f'--pad-type={pad_types[1]}', '--run-name=reflect padding')
# run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', f'--ct-window={windows[1]}', 
#     '--augmented', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}', f'--pad-type={pad_types[2]}', '--run-name=replicate padding')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', f'--ct-window={windows[1]}', 
    '--augmented', '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}', f'--pad-type={pad_types[2]}', '--run-name=temp run')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-11-14 23:03:42,017 INFO     pid:83547 nb:006:run Running: training.CovidSegmentationTrainingApp(['--num-workers=4', '--epochs=3', '--batch-size=32', '--ct-window=lung', '--augmented', '--width-irc', '7', '246', '246', '--pad-type=replicate', '--run-name=temp run']).main()
2020-11-14 23:03:43,213 INFO     pid:83547 training:195:init_model Using CUDA; 1 devices.
2020-11-14 23:03:45,388 INFO     pid:83547 modules.dsets:188:__init__ TrainingCovid2dSegmentationDataset: training mode, 177 uid's, 4379 index slices, 1080 lesions
2020-11-14 23:03:45,388 INFO     pid:83547 modules.dsets:229:__init__ TrainingCovid2dSegmentationDataset: (7, 246, 246) width_irc
2020-11-14 23:03:45,395 INFO     pid:83547 modules.dsets:188:__init__ Covid2dSegmentationDataset: validation mode, 20 uid's, 592 index slices, 157 lesions
2020-11-14 23:03:46,057 INFO     pid:83547 wandb:606:_apply_source_login setting login settings: {}
wandb: Currently logged in as: peterbacalso (use `wandb login --relogin` to force r

2020-11-14 23:03:47,561 INFO     pid:83547 training:478:main Starting CovidSegmentationTrainingApp, Namespace(augment_flip=False, augment_noise=False, augment_offset=False, augment_rotate=False, augment_scale=False, augmented=True, batch_size=32, ct_window='lung', depth=3, epochs=3, num_workers=4, pad_type='replicate', recall_priority=0, run_name='temp run', steps_per_epoch=10000.0, val_cadence=1, width_irc=['7', '246', '246'])


epoch,loss/trn,loss/val,metrics_val/miss_rate,metrics_val/fp_to_mask_ratio,pr_val/precision,pr_val/recall,pr_val/f1_score
1,0.5827,0.6085,0.1822,1.7875,0.3139,0.8178,0.4537
2,0.4744,0.5608,0.2053,1.2209,0.3943,0.7947,0.5271
3,0.4296,0.4847,0.2925,0.6352,0.5269,0.7075,0.6040


2020-11-14 23:04:39,840 INFO     pid:83547 training:316:log_metrics E1 CovidSegmentationTrainingApp
2020-11-14 23:04:39,842 INFO     pid:83547 training:345:log_metrics E1 trn      0.5827 loss, 0.3131 precision, 0.6745 recall, 0.4277 f1 score 0.3255 miss rate 1.4797 fp to label ratio
2020-11-14 23:04:57,182 INFO     pid:83547 training:316:log_metrics E1 CovidSegmentationTrainingApp
2020-11-14 23:04:57,184 INFO     pid:83547 training:354:log_metrics E1 val      0.6085 loss, 0.3139 precision, 0.8178 recall, 0.4537 f1 score 0.1822 miss rate 1.7875 fp to label ratio
2020-11-14 23:05:43,828 INFO     pid:83547 training:457:save_model Saved model params to saved-models/2020-11-14_23.03.43.10000.state
2020-11-14 23:05:43,830 INFO     pid:83547 training:462:save_model Saved model params to saved-models/2020-11-14_23.03.43.best.state
2020-11-14 23:05:43,833 INFO     pid:83547 training:467:save_model SHA1: acdcbacbc819269eb5baafa8af91af28825debdd
2020-11-14 23:06:36,871 INFO     pid:83547 training

loss/trn,0.42961
metrics_trn/miss_rate,0.34076
metrics_trn/fp_to_mask_ratio,0.47618
pr_trn/precision,0.58061
pr_trn/recall,0.65924
pr_trn/f1_score,0.61743
loss/val,0.48471
metrics_val/miss_rate,0.29251
metrics_val/fp_to_mask_ratio,0.6352
pr_val/precision,0.52692
pr_val/recall,0.70749


loss/trn,█▃▁
metrics_trn/miss_rate,▁██
metrics_trn/fp_to_mask_ratio,█▂▁
pr_trn/precision,▁▆█
pr_trn/recall,█▁▁
pr_trn/f1_score,▁▇█
loss/val,█▅▁
metrics_val/miss_rate,▁▂█
metrics_val/fp_to_mask_ratio,█▅▁
pr_val/precision,▁▄█
pr_val/recall,█▇▁


wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


RuntimeError: No active exception to reraise

# Train Run

In [4]:
epochs = 50
unet_depth=3
val_cadence = 5
batch_size = 32
recall_priority = 0
pad_type = 'replicate'
window = 'lung'
steps_per_epoch = 100000
bbox_dim = 246

In [5]:
run('training.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--ct-window={window}', '--augmented', f'--val-cadence={val_cadence}', f'--depth={unet_depth}', 
    '--width-irc', '7', f'{bbox_dim}', f'{bbox_dim}', f'--pad-type={pad_type}', 
    f'--steps-per-epoch={steps_per_epoch}', '--run-name=train run 1')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-11-14 23:02:20,392 INFO     pid:83413 nb:006:run Running: training.CovidSegmentationTrainingApp(['--num-workers=4', '--epochs=50', '--batch-size=32', '--ct-window=lung', '--augmented', '--val-cadence=5', '--depth=3', '--width-irc', '7', '246', '246', '--pad-type=replicate', '--steps-per-epoch=100000', '--run-name=train run 1']).main()
2020-11-14 23:02:21,544 INFO     pid:83413 training:195:init_model Using CUDA; 1 devices.
2020-11-14 23:02:24,153 INFO     pid:83413 modules.dsets:188:__init__ TrainingCovid2dSegmentationDataset: training mode, 177 uid's, 4379 index slices, 1080 lesions
2020-11-14 23:02:24,153 INFO     pid:83413 modules.dsets:229:__init__ TrainingCovid2dSegmentationDataset: (7, 246, 246) width_irc
2020-11-14 23:02:24,161 INFO     pid:83413 modules.dsets:188:__init__ Covid2dSegmentationDataset: validation mode, 20 uid's, 592 index slices, 157 lesions
2020-11-14 23:02:24,704 INFO     pid:83413 wandb:606:_apply_source_login setting login settings: {}
wandb: Currently lo

2020-11-14 23:02:26,106 INFO     pid:83413 training:478:main Starting CovidSegmentationTrainingApp, Namespace(augment_flip=False, augment_noise=False, augment_offset=False, augment_rotate=False, augment_scale=False, augmented=True, batch_size=32, ct_window='lung', depth=3, epochs=50, num_workers=4, pad_type='replicate', recall_priority=0, run_name='train run 1', steps_per_epoch=100000, val_cadence=5, width_irc=['7', '246', '246'])


epoch,loss/trn,loss/val,metrics_val/miss_rate,metrics_val/fp_to_mask_ratio,pr_val/precision,pr_val/recall,pr_val/f1_score


KeyboardInterrupt: 